In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from dateutil.parser import parse
import seaborn as sns
import numpy as np

In [ ]:
#importa datos el fluke
fluke = pd.read_csv("../../data/fluke_25062021_28062021.csv",encoding="iso-8859-1",
                    skiprows=13,usecols=[7,11],
                    names=["fluke","tiempo"],)
fluke.tiempo = pd.to_datetime(fluke.tiempo,format="%d/%m/%Y %H:%M")
fluke.set_index("tiempo",inplace=True)
fluke.fluke = fluke.fluke.str.replace("ppm","")
fluke.fluke = fluke.fluke.astype("float64")
fluke = fluke[~fluke.index.duplicated()]
#fluke.info()
# fluke["fluke_std"] = fluke.fluke.std()
fluke = fluke.resample("60S").mean()
fluke

In [ ]:
#importa datos del aranet4

aranet = pd.read_csv("../../data/aranet4_2021-06-28T12_08_22.csv",index_col=0,parse_dates=True,
                    names=["aranet","To","HR","P"],skiprows=1)

aranet = aranet.resample("60S").mean()
aranet

In [ ]:
camara = pd.read_csv("../../data/sensores_20210625_20210628.csv",index_col=0,parse_dates=True)
camara = camara.resample("60S").mean()
camara

In [ ]:
# camara.columns
sensores = ['t3022', 'mh-z14a', 'sen0219', 'cdm4160', 'cjmcu811', 'sen0220']

In [ ]:
data = pd.concat([fluke["fluke"],aranet.aranet,camara[sensores]],axis=1)

data.plot(subplots=True)
# data.aranet - data.sen0219

In [ ]:
data = pd.concat([fluke["fluke"],aranet.aranet,camara[sensores]],axis=1)

ref = "fluke"


fig, ax = plt.subplots(2,figsize=(20,10),sharex=True)

fecha1 = parse("2021-06-25 15:00") 
fecha2 = fecha1 + pd.Timedelta("2D 10H")


ax[0].set_ylim(300,2500)
ax[0].set_ylabel("CO2 [ppm]")
ax[0].set_title("Mediciones CO2")


ax[0].set_xlim(fecha1,fecha2)
ax[1].set_xlim(fecha1,fecha2)

ax[1].set_ylim(-1000,500)
ax[1].set_ylabel("Ref - Sensor CO2 [ppm]")
ax[1].set_xlabel("tiempo [mm-dd H]")
# ax[1].set_title("Mediciones CO2")

ax[0].plot(fluke.fluke,"ro",label="fluke",alpha=0.5)
for sensor in sensores:
    ax[0].scatter(data[sensor].index,data[sensor],label=sensor,marker=".")


for sensor in sensores:
    ax[1].scatter(data.index,data[ref]-data[sensor],label=sensor)


ax[1].legend()
ax[1].grid()
ax[0].legend()
ax[0].grid()





diferencia = {}
data = data[fecha1:fecha2]
for sensor in sensores:
    valor = (data[fecha1:fecha2][ref] - data[fecha1:fecha2][sensor]).mean()
    diferencia.update({sensor:[-valor]})


In [ ]:
ref = "fluke"
sensores = ['t3022', 'mh-z14a', 'sen0219', 'sen0220',"aranet"]
diferencia = pd.concat([data[sensor]-data[ref] for sensor in sensores],axis=1)
diferencia.columns = sensores

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))


ax.set_ylabel("Diferencia CO2 [ppm]")
ax.set_xlabel("Sensor [-]")


etiquetas = []
# diferencias = []
for i,sensor in enumerate(sensores):
    valor = ax.bar(i+1,(data[ref]-data[sensor]).mean() , yerr=(data[ref]-data[sensor]).std(), align='center')
    etiquetas.append(i+1)
    ax.bar_label(valor,fmt="%.0f")
    

ax.set_xticks(etiquetas)
ax.set_xticklabels(sensores)
ax.set_yticks(np.arange(-350,300,50))


ax.grid()
